<a href="https://colab.research.google.com/github/Joao-Girotto/Desafio-Python-Marvel/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalações

In [ ]:
!pip install matplot dotenv plotly

## Importações

In [ ]:
import requests
import hashlib
import os
import time
import pandas as pd
import sqlite3
from dotenv import load_dotenv
import regex as re
import matplotlib.pyplot as plt
import plotly.express as px


## Pegando as chaves

In [ ]:
load_dotenv()
PUBLIC_KEY = os.getenv("Public_Key")
PRIVATE_KEY = os.getenv("Private_Key")

## Endpoints e Parâmetros

In [ ]:
ts = str(time.time())
to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

end1 = "https://gateway.marvel.com/v1/public/characters"
end2 = "https://gateway.marvel.com/v1/public/comics"
end3 = "https://gateway.marvel.com/v1/public/books"
end4 = "https://gateway.marvel.com/v1/public/series"
end5 = "https://gateway.marvel.com/v1/public/creators"
end6 = "https://gateway.marvel.com/v1/public/events"
end7 = "https://gateway.marvel.com/v1/public/stories"

offset = 0
params = {
    "apikey": PUBLIC_KEY,
    "ts": ts,
    "hash": hash_md5,
    "limit": 100,
    "offset": offset
}

## Requisição

In [ ]:
response = requests.get(end1, params=params)

response
data = response.json()
data
#df = pd.DataFrame(data['data']['results'])
#df.dtypes
# df_extraido = pd.json_normalize(df['data']['results'])
# df_extraido2 = pd.DataFrame(df['data']['results'])

## Exibição

In [ ]:
# dsa = df_extraido['name']
df = df[['name', 'id', 'description']]
con = sqlite3.connect('Marvel.db')
df.to_sql("characters", con, if_exists="replace", index=False)
con.close()

# Função para consulta no BD

In [ ]:
def query_db(query, params=None, db='Marvel.db'):
  try:
    with sqlite3.connect(db) as con:
      return pd.read_sql_query(query, con, params)

  except Exception as e:
    print(f'Erro com a database: {e}')
    return pd.DataFrame()


# Função para fazer as queries


In [ ]:
def query_api(end, offset=0):
  arraySeries = []
  params['offset'] = offset
  max_tries = 5
  while True:
    tries = 0

    while tries < max_tries:
      try:
        response = requests.get(end,params=params)
        response.raise_for_status()
        data = response.json()

        if data.get('data', {}).get('results', []):
          arraySeries.append(data)
          break

        else:
          print('Failed, trying again')
          tries += 1
          continue

      except Exception as e:
        print('Trying again, HTTP status code != 200')
        tries += 1
        continue

    if tries == max_tries:
        print('Offset failed, jumping offset')
        offset += 100
        tries = 0
        params['offset'] = offset

    offset += 100
    tries = 0
    params['offset'] = offset
    print(offset)

    if offset >= data.get('data', {}).get('total', []):
      print('Query finished with success')
      break
  return arraySeries

# Função para criar um CSV

In [ ]:
def create_csv(array, nome_arquivo): ## Array de JSON

  try:
    df = pd.DataFrame(array)
    df.to_csv(nome_arquivo, index=False, encoding='utf-8')

  except Exception as e:
    print(f'Error when creating the file {nome_arquivo}: {e}')

# Função para plotar um gráfico

In [ ]:
def plot(dataframe ,title, x_label, y_label, grid='y', kind='bar', color='purple', rotation=90, show=True, **kwargs):
  ax = dataframe.plot(kind=kind, color=color, title=title, **kwargs)
  ax.set_xlabel(x_label)
  ax.set_ylabel(y_label)
  plt.xticks(rotation=rotation)
  plt.tight_layout()
  plt.grid(axis=grid)

  if show:
    plt.show()

  return ax

# Series

## Adicionando as series ao banco de dados

In [ ]:
def series_add_db(array): ## Deve ser um array contendo o JSON de response, cada índice é um JSON, NÃO remover o header do response para passar pra função
  def calculateDuration(row):
    if row['endYear'] == 2099:
      return 999
    else:
      return row['endYear'] - row['startYear']

  def function_dict(x):
    creators = [item['name'] for item in x['items']] if isinstance(x, dict) and 'items' in x else []
    return ', '.join(creators)

  def amount(x):
    return x.get('available', 0)

  with sqlite3.connect('Marvel.db') as con:
    for i in array:
      df = pd.DataFrame(pd.DataFrame(i['data']['results']))
      df['duration'] = df.apply(calculateDuration, axis=1)
      df['creator_name'] = df['creators'].apply(function_dict)
      df['character_qty'] = df['characters'].apply(amount)
      df['creator_qty'] = df['creators'].apply(amount)
      df['story_qty'] = df['stories'].apply(amount)
      df['event_qty'] = df['events'].apply(amount)
      df['comic_qty'] = df['comics'].apply(amount)
      df = df[['id', 'title', 'description', 'startYear', 'endYear',
              'rating', 'type', 'duration', 'creator_name', 'creator_qty',
              'character_qty', 'story_qty', 'event_qty', 'comic_qty']]
      df.to_sql("series", con, if_exists="append", index=False)

## Função para analisar a quatidade de séries por ano

In [ ]:
def series_by_year(dataframe, show_plot=False):
  series_year = dataframe[(dataframe['startYear'] > 0) & (dataframe['startYear'] < 2029)]['startYear'].value_counts().sort_index()

  if show_plot:
    plot(dataframe=series_year, title='Séries por Década', x_label='Década',
         y_label='Quantidade de Séries', figsize=(20, 6))

  return series_year

## Função para analisar a média de duração das séries

In [ ]:
def series_average_duration(dataframe, show_plot=False):
  anos_series = dataframe[(dataframe['duration'] >= 0) &(dataframe['duration'] < 999)]['duration'].value_counts().sort_index().head(10) ## 999 definido para séries em produção

  if show_plot:
    plot(dataframe=anos_series, title='Series: Média da duração das series', x_label='Duração (anos)',
         y_label='Quantidade de séries')

  media = dataframe[(dataframe['duration'] >= 0) &(dataframe['duration'] < 999)]['duration'].mean() ## 999 definido para séries em produção
  return anos_series, media

## Função para analisar a distribuição das classificações

In [ ]:
def series_distribution_rating(dataframe, show_plot=False):
  media = dataframe[dataframe['rating'] != '']['rating'].value_counts()

  if show_plot:
    plot(dataframe=media, x_label='Classificação etária', y_label='Quantidade de séries',
         title='Distribuição classificatória de faixa etária')

  return media

## Função para analisar a distribuição dos tipos

In [ ]:
def series_distribuition_type(dataframe, show_plot=False):
  series_type = dataframe[dataframe['type'] != '']['type'].value_counts()

  if show_plot:
    plot(dataframe=series_type, x_label='Tipo', title='Distribuição dos tipos de séries',
         y_label='Quantidade de séries')

  return series_type

## Função para analisar a quantidade de personagens por ano

In [ ]:
def series_quantity_characters_year(dataframe, show_plot=False):
  tempo = dataframe[(dataframe['character_qty'] > 0) & (dataframe['startYear'] > 1900) & (dataframe['startYear'] < 2029)].groupby('startYear')['character_qty'].sum()
  falta_anos = pd.Series(0, index=range(1939, 2025))
  arrumado =  falta_anos.add(tempo, fill_value=0)

  if show_plot:
    plot(dataframe=arrumado, title='Quantidade de personagens por ano', x_label='Ano', y_label='Quantidade de personagens', figsize=(20, 6))

  return arrumado

In [ ]:
array = query_api(end4)
create_csv(array=array, nome_arquivo='Series.csv')
series_add_db(array=array)

##Pegando do banco de dados

In [ ]:
data = query_db('SELECT * FROM series')

## INSIGHT 1 Series: Quantidade de issues lançadas por ano

In [ ]:
series_by_year(data, show_plot=True)

## INSIGHT 2 Series: Média da duração das series

In [ ]:
series_average_duration(data, show_plot=True)

## INSIGHT 3 Series: Distribuição classificatória de faixa etária

In [ ]:
series_distribution_rating(data, show_plot=True)

## INSIGHT 4 Series: Distribuição dos tipos de séries

In [ ]:
series_distribuition_type(data, show_plot=True)

## INSIGHT 5 Series: Quantidade de personagens por ano

In [ ]:
series_quantity_characters_year(data, show_plot=True)

## Dados errados da API em relação a duração

In [ ]:
duration = data['duration'].value_counts().sort_index()
data[(data['duration'] < 0) | (data['duration'] > 999)]

# Comics

## Função para adicionar as comics no banco de dados

In [ ]:
def comics_add_db(array_json): ## Deve ser um array contendo o JSON de response, cada índice é um JSON, NÃO remover o header do response para passar pra função

  def funcao_series(x):
    return x.get('name', '') if isinstance(x, dict) else ''

  def funcao_price(x):
      if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict):
          return x[0].get('price', '')
      return ''

  with sqlite3.connect('Marvel.db') as con:
    for i in array_json:
      df = pd.DataFrame(pd.DataFrame(i['data']['results']))
      df['price'] = df['prices'].apply(funcao_price)
      df['series_name'] = df['series'].apply(funcao_series)
      df = df[['id', 'digitalId','title', 'issueNumber', 'description',
              'format', 'pageCount', 'series_name', 'price']]
      df.to_sql("comics", con, if_exists="append", index=False)



## Função para contar a quatidade de cada formato

In [ ]:
def comics_count_format(dataFrame, show_plot=False): ## O DataFrame deve ser o DataFrame pós-adição no banco de dados, com os headers HTTP já removidos
  format = dataFrame[dataFrame['format'] != '']['format'].value_counts()

  if show_plot:

    fig = px.bar(
        format,
        x=format.index,
        y=format.values,
        labels={'x': 'Formato', 'y': 'Quantidade de comics'},
        title='Quantidade de cada tipo',
        color_discrete_sequence=['purple']
    )

    fig.update_layout(
        xaxis_title='Formato',
        yaxis_title='Quantidade de comics',
        xaxis_tickangle=-45
    )

    fig.show()

  return format

## Função para contar a média de preço das comics sobre o tempo

In [ ]:
def comics_average_price_over_time(dataframe, show_plot=False): ## O DataFrame deve ser o DataFrame pós-adição no banco de dados, com os headers HTTP já removidos
  def extract_number(text):
    match = re.findall(r'\d+', text)
    return int(match[0]) if match else None

  dataframe['year'] = dataframe['title'].apply(extract_number)
  price_over_time = dataframe[['year', 'price']]
  price_over_time = price_over_time[(price_over_time['year'] > 1900) & (price_over_time['year'] < 2050)]
  price_filter = price_over_time.query('price > 0 and price < 20').copy()
  price_filter = price_filter.dropna()
  price_filter_average = price_filter.groupby('year').mean()

  if show_plot:
    plt.figure(figsize=(20, 6))
    plt.plot(price_filter_average.index, price_filter_average['price'], color='purple', label='Preço Médio')
    plt.scatter(price_filter['year'], price_filter['price'], color='grey', label='Preços individuais', alpha=0.4, s=10)

  return price_filter_average

## Função para contar a média de páginas das comics sobre o tempo

In [ ]:
def comics_average_pages_over_time(dataframe, show_plot=False): ## O DataFrame deve ser o DataFrame pós-adição no banco de dados, com os headers HTTP já removidos
  def extract_number(text):
    match = re.findall(r'\d+', text)
    return int(match[0]) if match else None

  dataframe['year'] = dataframe['title'].apply(extract_number)
  pages_over_time = dataframe[['year', 'pageCount']]
  pages_over_time = pages_over_time[(pages_over_time['year'] > 1900) & (pages_over_time['year'] < 2050)]
  pages_filter = pages_over_time.query('pageCount > 0 and pageCount < 200')
  pages_filter = pages_filter.dropna()
  pags_filter_average = pages_filter.groupby('year').mean()

  if show_plot:
    plot(dataframe=pags_filter_average, title='Média de páginas', x_label='Ano', y_label='Média de páginas', kind='line')

  return pags_filter_average

## Consumindo e armazenando os dados

In [ ]:
array = query_api(end2) ## Endereço de comics
create_csv(array, "Comics.csv")
comics_add_db(array)

## Pegando os dados do banco de dados

In [ ]:
data = query_db('SELECT * FROM comics')

## INSIGHT 1 Comics: Média de preço sobre o tempo

In [ ]:
comics_average_price_over_time(data, show_plot=True)

## INSIGHT 2 Comics: Média de páginas sobre o tempo

In [ ]:
comics_average_pages_over_time(data, show_plot=True)

## INSIGHT 3 Comics: Os formatos mais comuns

In [ ]:
comics_count_format(data, show_plot=True)

# Stories

## Função para adicionar os stories ao BD

In [ ]:
def stories_add_db(array_json): ## Deve ser um array contendo o JSON de response, cada índice é um JSON, NÃO remover o header do response para passar pra função
  with sqlite3.connect('Marvel.db') as con:
    for i in array_json:
      df4 = pd.DataFrame(pd.DataFrame(i['data']['results']))
      df4 = df4[['id', 'title', 'description', 'type']]
      df4.to_sql("stories", con, if_exists="append", index=False)


## Função para calcular a quantidade de cada tipo de stories

In [ ]:
def stories_quantity_type(dataframe, show_plot=False): ## O DataFrame deve ser o DataFrame pós-adição no banco de dados, com os headers HTTP já removidos
  types = dataframe[dataframe['type'] != '']['type'].value_counts()

  if show_plot:

    fig = px.bar(
        types,
        x=types.index,
        y=types.values,
        labels={'x': 'Tipo', 'y': 'Quantidade'},
        title='Quantidade de cada tipo',
        color_discrete_sequence=['purple']
    )

    fig.update_layout(
        xaxis_title='Tipos',
        yaxis_title='Quantidade',
        xaxis_tickangle=-45
    )

    fig.show()

  return types

## Consumindo e armazenando os dados

In [ ]:
array = query_api(end7)
create_csv(array, "Stories.csv")
stories_add_db(array)

## Pegando os dados do banco de dados

In [ ]:
dataframe = query_db("SELECT * FROM stories")

## INSIGHT 1 Stories: Quantidade de cada tipo




In [ ]:
types = stories_quantity_type(dataframe, show_plot=True)
